# all_times_aid_features

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
import pickle
from collections import defaultdict

import pandas as pd
import cudf
import matplotlib.pyplot as plt

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
week = "week3"

In [5]:
# データ読み込み
files = [
    "train_sessions_week1.pkl",
    "test_sessions_week1.pkl",
    "train_sessions_week2.pkl",
    "test_sessions_week2.pkl",
    "train_sessions_week3.pkl",
 #   "test_sessions_week3.pkl",
    "train_sessions_week4.pkl",
    "test_sessions_week4.pkl",
    "test_sessions.pkl"
]

dfs = []

for file in files:
    dfs.append(pd.read_pickle(PREP_DIR + file))
sessions = pd.concat(dfs)

all_times_aid_freq

In [6]:
result = sessions.groupby("aid")["session"].count().to_dict()
with open(PREP_DIR + f"all_times_aid_freq_{week}.pkl", "wb") as f:
    pickle.dump(result, f)

all_times_aid_{type}_freq

In [7]:
for type_ in ["clicks", "carts", "orders"]:
    result = sessions[sessions["type"]==type_].groupby("aid")["session"].count().to_dict()
    with open(PREP_DIR + f"all_times_aid_{type_}_freq_{week}.pkl", "wb") as f:
        pickle.dump(result, f)

all_times_aid_session_freq

In [8]:
sessions_drop_dup = sessions.drop_duplicates(["session", "aid"])
result = sessions_drop_dup.groupby("aid")["session"].count().to_dict()
with open(PREP_DIR + f"all_times_aid_session_freq_{week}.pkl", "wb") as f:
    pickle.dump(result, f)

all_times_aid_session_{type}_freq

In [9]:
sessions_drop_dup = sessions.drop_duplicates(["session", "aid", "type"])
for type_ in ["clicks", "carts", "orders"]:
    result = sessions_drop_dup[sessions_drop_dup["type"]==type_].groupby("aid")["session"].count().to_dict()
    with open(PREP_DIR + f"all_times_aid_session_{type_}_freq_{week}.pkl", "wb") as f:
        pickle.dump(result, f)

all_times_aid_{type}_ratio_with_clicks

In [10]:
result = pd.DataFrame(sessions["aid"].unique(), columns=["aid"])
for type_ in ["clicks", "carts", "orders"]:
    with open(PREP_DIR + f"all_times_aid_session_{type_}_freq_{week}.pkl", "rb") as f:
        result[type_] = result["aid"].map(pickle.load(f))
result = result.fillna(0)
result = result[result["clicks"]>0]

result["all_times_aid_carts_ratio_with_clicks"] = result["carts"] / result["clicks"]
result["all_times_aid_orders_ratio_with_clicks"] = result["orders"] / result["clicks"]

with open(PREP_DIR + f"all_times_aid_carts_ratio_with_clicks_{week}.pkl", "wb") as f:
    pickle.dump(result["all_times_aid_carts_ratio_with_clicks"].to_dict(), f)

with open(PREP_DIR + f"all_times_aid_orders_ratio_with_clicks_{week}.pkl", "wb") as f:
    pickle.dump(result["all_times_aid_orders_ratio_with_clicks"].to_dict(), f)

all_times_aid_{dt}_std

In [11]:
sessions["dt"] = pd.to_datetime(sessions["ts"], unit="ms")

sessions["date"] = sessions["dt"].dt.date
sessions["hour"] = sessions["dt"].dt.hour
sessions["day_of_week"] = sessions["dt"].dt.day_of_week

for dt_ in ["date", "hour", "day_of_week"]:
    agg_df = sessions.groupby(["aid", dt_])["session"].count().reset_index()
    result = agg_df.groupby("aid")["session"].std().to_dict()

    with open(PREP_DIR + f"all_times_aid_{dt_}_std_{week}.pkl", "wb") as f:
        pickle.dump(result, f)